In [3]:
## imports 

# For testing detectron2 mask r-cnn with COCO 

import detectron2 

#Setup logger
from detectron2.utils.logger import setup_logger
setup_logger()

#Some common libraries 
import numpy as np 
import os,json,cv2,random 


#Detectron2 utilities 

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg 
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog


/Users/HeleneSemb/opt/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import sys 
sys.path.append(r"/Users/HeleneSemb/Documents/chipsogdip/kode")
from kode.training import *
from kode.dataset import * 
from detectron2.structures import BoxMode 
from detectron2.utils.visualizer import ColorMode 

cfg = config() 

cfg.OUTPUT_DIR = "/Volumes/helensem/Master/output/run1/resnet50"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
cfg.MODEL.DEVICE = "cpu"


predictor = DefaultPredictor(cfg) 
damage_metadata = MetadataCatalog.get("damage_train")

dataset_dicts = load_damage_dicts(r"/Volumes/helensem/Master/Labeled_pictures", "val")
for d in random.sample(dataset_dicts, 3): 
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                    metadata = damage_metadata,
                    scale = 0.5,
                    instance_mode = ColorMode.IMAGE_BW)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imwrite(f"Volumes/helensem/Master/predictions/{d['file_name']}", out.get_image()[:,:,::-1]) #* Save images 
    #cv2.imshow("image",out.get_image()[:,:,::-1])
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

[02/23 13:23:20 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Volumes/helensem/Master/output/run1/resnet50/model_final.pth ...
[02/23 13:23:20 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Volumes/helensem/Master/output/run1/resnet50/model_final.pth ...
/Volumes/helensem/Master/Labeled_pictures/val/DSC00224
/Volumes/helensem/Master/Labeled_pictures/val/DSC00224/1.png
/Volumes/helensem/Master/Labeled_pictures/val/DSC00224/masks/grov_merking_0.png
/Volumes/helensem/Master/Labeled_pictures/val/7
/Volumes/helensem/Master/Labeled_pictures/val/7/7.jpeg
/Volumes/helensem/Master/Labeled_pictures/val/7/masks/grov_merking_6.png
Contour too small:  /Volumes/helensem/Master/Labeled_pictures/val/7/masks/grov_merking_6.png
/Volumes/helensem/Master/Labeled_pictures/val/7/masks/grov_merking_7.png
/Volumes/helensem/Master/Labeled_pictures/val/7/masks/grov_merking_4.png
/Volumes/helensem/Master/Labeled_pictures/val/7/masks/grov_merking_2.png
/Vol

libpng warning: iCCP: known incorrect sRGB profile


/Volumes/helensem/Master/Labeled_pictures/val/000570/masks/grov_merking_0.png
/Volumes/helensem/Master/Labeled_pictures/val/000570/masks/grov_merking_1.png
/Volumes/helensem/Master/Labeled_pictures/val/19
/Volumes/helensem/Master/Labeled_pictures/val/19/19.jpeg
/Volumes/helensem/Master/Labeled_pictures/val/19/masks/grov_merking_4.png
/Volumes/helensem/Master/Labeled_pictures/val/19/masks/grov_merking_2.png
/Volumes/helensem/Master/Labeled_pictures/val/19/masks/grov_merking_1.png
/Volumes/helensem/Master/Labeled_pictures/val/19/masks/grov_merking_5.png
/Volumes/helensem/Master/Labeled_pictures/val/19/masks/grov_merking_3.png
/Volumes/helensem/Master/Labeled_pictures/val/ck686hjjsf3za0842m6qkgjlv
/Volumes/helensem/Master/Labeled_pictures/val/ck686hjjsf3za0842m6qkgjlv/6098042jpg.png
/Volumes/helensem/Master/Labeled_pictures/val/ck686hjjsf3za0842m6qkgjlv/masks/corrosion_8.png
/Volumes/helensem/Master/Labeled_pictures/val/ck686hjjsf3za0842m6qkgjlv/masks/corrosion_1.png
/Volumes/helensem/Mas

libpng warning: iCCP: known incorrect sRGB profile


/Volumes/helensem/Master/Labeled_pictures/val/000557/masks/grov_merking_0.png
/Volumes/helensem/Master/Labeled_pictures/val/ck686asjfeetl0833zoesh2l7
/Volumes/helensem/Master/Labeled_pictures/val/ck686asjfeetl0833zoesh2l7/8996659jpg.png
/Volumes/helensem/Master/Labeled_pictures/val/ck686asjfeetl0833zoesh2l7/masks/corrosion_1.png
/Volumes/helensem/Master/Labeled_pictures/val/ck686asjfeetl0833zoesh2l7/masks/corrosion_5.png
/Volumes/helensem/Master/Labeled_pictures/val/ck686asjfeetl0833zoesh2l7/masks/corrosion_2.png
/Volumes/helensem/Master/Labeled_pictures/val/ck686asjfeetl0833zoesh2l7/masks/corrosion_3.png
/Volumes/helensem/Master/Labeled_pictures/val/ck686asjfeetl0833zoesh2l7/masks/corrosion_0.png
/Volumes/helensem/Master/Labeled_pictures/val/ck686asjfeetl0833zoesh2l7/masks/corrosion_4.png
/Volumes/helensem/Master/Labeled_pictures/val/ck686asjfeetl0833zoesh2l7/masks/corrosion_6.png
/Volumes/helensem/Master/Labeled_pictures/val/DSC00223
/Volumes/helensem/Master/Labeled_pictures/val/DSC0

...

/Volumes/helensem/Master/Labeled_pictures/val/000508/masks/grov_merking_0.png
/Volumes/helensem/Master/Labeled_pictures/val/000481
/Volumes/helensem/Master/Labeled_pictures/val/000481/1.png
/Volumes/helensem/Master/Labeled_pictures/val/000481/masks/grov_merking_0.png
/Volumes/helensem/Master/Labeled_pictures/val/IMG_3391
/Volumes/helensem/Master/Labeled_pictures/val/IMG_3391/1.png
/Volumes/helensem/Master/Labeled_pictures/val/IMG_3391/masks/grov_merking_0.png
/Volumes/helensem/Master/Labeled_pictures/val/IMG_3391/masks/grov_merking_6.png
/Volumes/helensem/Master/Labeled_pictures/val/IMG_3391/masks/grov_merking_4.png
/Volumes/helensem/Master/Labeled_pictures/val/IMG_3391/masks/grov_merking_2.png
/Volumes/helensem/Master/Labeled_pictures/val/IMG_3391/masks/grov_merking_1.png
/Volumes/helensem/Master/Labeled_pictures/val/IMG_3391/masks/grov_merking_5.png
/Volumes/helensem/Master/Labeled_pictures/val/IMG_3391/masks/grov_merking_3.png
/Volumes/helensem/Master/Labeled_pictures/val/ck736olf09

libpng warning: iCCP: known incorrect sRGB profile


/Volumes/helensem/Master/Labeled_pictures/val/000541/masks/grov_merking_0.png
/Volumes/helensem/Master/Labeled_pictures/val/ck739a2thahot0839rq2kmh6z
/Volumes/helensem/Master/Labeled_pictures/val/ck739a2thahot0839rq2kmh6z/6086846jpg.png
/Volumes/helensem/Master/Labeled_pictures/val/ck739a2thahot0839rq2kmh6z/masks/corrosion_1.png
/Volumes/helensem/Master/Labeled_pictures/val/ck739a2thahot0839rq2kmh6z/masks/corrosion_2.png
/Volumes/helensem/Master/Labeled_pictures/val/ck739a2thahot0839rq2kmh6z/masks/corrosion_3.png
/Volumes/helensem/Master/Labeled_pictures/val/ck739a2thahot0839rq2kmh6z/masks/corrosion_0.png
/Volumes/helensem/Master/Labeled_pictures/val/ck739a2thahot0839rq2kmh6z/masks/corrosion_4.png
/Volumes/helensem/Master/Labeled_pictures/val/ck738sgeva81q0911ao8npamy
/Volumes/helensem/Master/Labeled_pictures/val/ck738sgeva81q0911ao8npamy/6074276jpg.png
/Volumes/helensem/Master/Labeled_pictures/val/ck738sgeva81q0911ao8npamy/masks/corrosion_8.png
/Volumes/helensem/Master/Labeled_picture

[ WARN:0@78384.552] global loadsave.cpp:244 findDecoder imread_('/Volumes/helensem/Master/Labeled_pictures/val/000550/1.png'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [33]:
from detectron2.structures import BoxMode 


def find_contours(sub_mask):
    assert sub_mask is not None, "file could not be read, check with os.path.exists()"
    imgray = cv2.cvtColor(sub_mask, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(imgray, 127, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours[0]


def create_image_annotation(file_name, width, height, image_id):
    return {
        "id": image_id,
        "width": width,
        "height": height,
        "file_name": file_name,
    }


def create_annotation_format(contour):
    return {
        "iscrowd": 0,
        "segmentation": [contour.flatten().tolist()],
        "bbox": cv2.boundingRect(contour),
        "bbox_mode": BoxMode.XYWH_ABS,
        "category_id": 0,
    }

In [36]:
import skimage


def load_damage_dicts(dataset_dir, subset): 
    dataset_dicts = []

    assert subset in ["train", "val"]
    dataset_dir = os.path.join(dataset_dir, subset)
    image_ids = next(os.walk(dataset_dir))[1]
    for image_id in image_ids:

        image_dir = os.path.join(dataset_dir, image_id)
        (_, _, file_names) = next(os.walk(image_dir))
        file_name = file_names[0]
        image_path = os.path.join(image_dir, file_name)

        height, width = cv2.imread(image_path).shape[:2]
        record = create_image_annotation(image_path, width, height, image_id)

        mask_dir = os.path.join(image_dir, 'masks')
        objs = []
        for f in next(os.walk(mask_dir))[2]:
            if f.endswith('.png') and ('corrosion' or 'grov_merking' in f):
                mask_path = os.path.join(mask_dir, f)
                mask = cv2.imread(mask_path)
                contour = find_contours(mask)
                obj = create_annotation_format(contour)
                objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts


for d in ["train", "val"]:
    DatasetCatalog.register("damage_" + d, lambda d=d: load_damage_dicts(r"/Users/HeleneSemb/Documents/Master/chipsogdip", "train"))
    MetadataCatalog.get("damage_" + d).set(thing_classes=["damage"])

balloon_metadata = MetadataCatalog.get("damage_train")

In [ ]:
dataset_dicts = load_damage_dicts(r"/Users/HeleneSemb/Documents/Master/chipsogdip", "train")
for d in random.sample(dataset_dicts, 1): 
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:,:,::-1], metadata = balloon_metadata, scale =0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2.imshow("imageout", out.get_image()[:,:,::-1])
    cv2.waitKey(0)
    # closing all open windows
    cv2.destroyAllWindows()

In [2]:
%load_ext tensorboard
%tensorboard --logdir /Volumes/helensem/Master/output/run1/resnet50


Reusing TensorBoard on port 6006 (pid 6407), started 0:14:21 ago. (Use '!kill 6407' to kill it.)